In [ ]:
!pip install --upgrade pandas numpy dask black[jupyter] uproot3 pyarrow astropy toml

In [ ]:
import uproot3

import dask.dataframe as dd
import pandas as pd
import numpy as np
from astropy import units as u
import matplotlib.pyplot as plt
from dataclasses import dataclass

%matplotlib inline

In [ ]:
!ls -lah *.root

In [ ]:
import tdat

In [ ]:
input_files = !ls data*.root

In [ ]:
input_files

In [ ]:
dask_dataframe = dd.from_map(tdat.read_root_file, input_files)

In [ ]:
dask_dataframe.head()

In [ ]:
dask_dataframe.value1.mean().compute()

In [ ]:
len(dask_dataframe.value1)

In [ ]:
type(_)

In [ ]:
# dask_dataframe.to_parquet

In [ ]:
tdat.dask_memory_usage_GB(dask_dataframe)

In [ ]:
centroids, histogram = tdat.dask_histogram(
    dask_dataframe, "value1", bins=20, bins_range=[-5, 5]
)

In [ ]:
plt.bar(centroids, histogram.compute())
plt.grid();

In [ ]:
from tdat.filters import ComputeColumn, QueryFilter

In [ ]:
ComputeColumn??

In [ ]:
func = ComputeColumn("value1", "value2")

In [ ]:
dask_dataframe_added_column = dask_dataframe.map_partitions(func)

In [ ]:
filt = QueryFilter("computed_col < 1")

In [ ]:
QueryFilter??

In [ ]:
dask_dataframe_added_column_filtered = dask_dataframe_added_column.map_partitions(filt)

In [ ]:
output_pandas_dataframe = dask_dataframe_added_column_filtered.compute()

In [ ]:
# filters also run on pandas dataframes

filt(output_pandas_dataframe).head()

In [ ]:
len(output_pandas_dataframe) / 1e6

## Execute multiple computations at the same time

No computation is executed when calling `dask_histogram`,
computation is triggered by `dd.compute` and input files are read 1 once.

Also, common part of the computation diagram are executed once.

In [ ]:
centroids, histogram_squaredsum = tdat.dask_histogram(
    dask_dataframe_added_column, "value1", bins=20, bins_range=[-5, 5]
)

centroids, histogram_filtered = tdat.dask_histogram(
    dask_dataframe_added_column_filtered, "value1", bins=20, bins_range=[-5, 5]
)

In [ ]:
h1,h2,h3 = dd.compute(histogram, histogram_squaredsum, histogram_filtered)

In [ ]:
for i, h in enumerate([h1, h2, h3]):
    plt.subplot(1, 3, i+1)
    plt.bar(centroids, h)
    plt.grid();

## Use the configuration files

In [ ]:
%%file filters/datasetname/compute_squaredsum.toml

class = "ComputeColumn"
column_1 = "value1"
column_2 = "value2"

In [ ]:
%%file filters/datasetname/filter_computedcol.toml

class = "QueryFilter"
query = "computed_col < 1"

In [ ]:
compute_squaredsum = tdat.load_filter("filters/datasetname/compute_squaredsum.toml")

In [ ]:
dask_dataframe.map_partitions(compute_squaredsum).head()

In [ ]:
tdat.load_filter("filters/datasetname/filter_computedcol.toml")

## Use Pipeline

In [ ]:
%%file filters/datasetname/dataset_pipeline.toml

class = "Pipeline"
steps = ["filters/datasetname/compute_squaredsum.toml", "filters/datasetname/filter_computedcol.toml"]

In [ ]:
pipeline = tdat.load_filter("filters/datasetname/dataset_pipeline.toml")

In [ ]:
pipeline.filters

In [ ]:
pipeline.filters[1].query

In [ ]:
dask_dataframe_cleaned = pipeline.apply(dask_dataframe)

In [ ]:
dask_dataframe_cleaned.head()